In [1]:
import os
import dask.dataframe as dd
import featuretools as ft
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
path_profile = r'D:\issp_data\zok_ssp_2017.csv'

log_root = r'D:\issp_data\fe_log_new'
out_root = r'D:\issp_data\features_total_final'

In [3]:
usecols = [
    '世帯年収', '居住都道府県(参考)', 
#     '最終学歴',  # 要做成dummy變數, 只要這個就好  
    '1ヶ月平均小遣い', '年齢(5才刻み)', '家屋形態', '居住エリア', '職種', '個人年収',
    '世代', '家族人数', '職業(詳細)', '仕事環境', '生協加入有無', 
    'メディアターゲット区分',
    '家族構成', '自動車(自家用車保有)有無', '性年代', '同居配偶者職業',
    '同居配偶者年齢(5才刻み)', '性別', '未既婚'
]
print(len(usecols))

# dropcols = [
#     'モニターID', '暗号化ID(40桁)', '居住都道府県(参考)', '居住エリア', 
#     '年齢(5才刻み)_名称', '年齢(10才刻み)', '年齢(10才刻み)_名称', '世代', 
#     '性別_名称', '性年代', 'メディアターゲット区分', '未既婚_名称', '職業', '職業_名称', 
#     '職業(詳細)', '職種', '仕事環境', '個人年収_名称', '1ヶ月平均小遣い_名称', 
#     '最終学歴_名称', '家族人数_名称', '家族構成', '世帯年収_名称', '家屋形態',
#     '生協加入有無_名称', '配偶者の同居有無', '同居配偶者年齢(5才刻み)_名称', 
#     '同居配偶者年齢(10才刻み)', '同居配偶者年齢(10才刻み)_名称', '同居配偶者職業', 
#     '末子年齢_名称', '家族人員(17才以下男子)_名称', '家族人員(17才以下女子)_名称', 
#     '家族人員(0〜2才(乳児))_名称', '家族人員(3〜5才(就学前児童))_名称', 
#     '家族人員(6〜11才(小学生))_名称', '家族人員(12〜14才(中学生))_名称', 
#     '家族人員(15〜17才(高校生))_名称', '家族人員(65才以上)_名称', 
#     '自動車(自家用車保有)有無_名称', '同居子供人数（17才以下）_名称', 
#     '同居子供人数（14才以下）_名称', 'ウエイトセル', 'ウエイトセル_名称'
# ]

# dummy = [
#     '居住都道府県(参考)_名称',
#     '居住エリア_名称',
#     '世代_名称',  # 之所以讓這個本來可以catagorical的東西變成dummy, 是因為它跟年齡都是連續的, 有鑑於性質重複所以特別改了
#     '性年代_名称',
#     'メディアターゲット区分_名称',
#     '職業(詳細)_名称',
#     '職種_名称', 
#     '仕事環境_名称',
#     '家族構成_名称',
#     '家屋形態_名称',
#     '配偶者の同居有無_名称',
#     '同居配偶者職業_名称'
# ]

columns_jp2en = {
    r'モニターID(11桁)': r'Monitor_ID'
}

df_dummy = pd.get_dummies(
                pd.read_csv(path_profile, encoding='shift-jis', 
                            usecols=['モニターID(11桁)', '最終学歴_名称'])\
                    .rename(columns=columns_jp2en)\
                    .set_index('Monitor_ID'))
print(df_dummy.shape)

df_profile = df_dummy.join(
                pd.read_csv(path_profile, encoding='shift-jis', 
                            usecols=usecols + ['モニターID(11桁)'])\
                    .rename(columns=columns_jp2en)\
                    .set_index('Monitor_ID')
                , on='Monitor_ID')
print(df_profile.shape)

21
(33932, 8)
(33932, 29)


In [4]:
# # adjust offset of categorical variables
# categorical = [
#     '年齢(5才刻み)',  # 要減1, 調整offset --> F1
#     '性別',  # 要減1, 其實這個categorical或dummy都可以, 因為是二元分類嘛, 但避免column數過多, 就變成cate vars --> F1
#     '未既婚',  # 這也要減1, 理由同上 --> F1
#     '個人年収',  # return 0 if x == 10 else x --> 如果 x == 10 反而是沒收入, 把它歸零, 其餘不變 --> F2
#     '1ヶ月平均小遣い',  # return 0 if x == 10 else x - 1 --> 10是不明, 1是沒收入(要將它規0) --> F3
    
#     '最終学歴',  # return 0 if x == 8 else x --> F4
#     '家族人数',  # 不需要任何調整, good! --> None
#     '世帯年収',  # return 0 if x == 6 else x --> F5
#     '生協加入有無',  # 要減1, 調整offset --> F1
#     '同居配偶者年齢(5才刻み)',  # return 0 if x == 10 else x, 因為x == 10表示沒有配偶/沒有同居 --> F2
    
#     '末子年齢',  # 要減1, 調整offset, 因為x==1是表示沒小孩= = --> F1
#     '家族人員(17才以下男子)',  # 要減1, 調整offset
#     '家族人員(17才以下女子)',  # 同上
#     '家族人員(0〜2才(乳児))',  # 同上
#     '家族人員(3〜5才(就学前児童))',  # 同上
    
#     '家族人員(6〜11才(小学生))',  # 同上
#     '家族人員(12〜14才(中学生))',  # 同上
#     '家族人員(15〜17才(高校生))',  # 同上
#     '家族人員(65才以上)',  # 同上
#     '自動車(自家用車保有)有無',  # 同上
    
#     '同居子供人数（17才以下）',  # 同上, 要減1, 調整offset
#     '同居子供人数（14才以下）'  # 同上, 要減1, 調整offset
# ]

# F1 = lambda x: x - 1
# F2 = lambda x: 0 if x == 10 else x
# F3 = lambda x: 0 if x == 10 else x - 1
# F4 = lambda x: 0 if x ==  8 else x
# F5 = lambda x: 0 if x ==  6 else x

# func_list = [F1, F1, F1, F2, F3, 
#              F4, None, F5, F1, F2, 
#              F1, F1, F1, F1, F1, 
#              F1, F1, F1, F1, F1,
#              F1, F1]

# df_cate = pd.read_csv(path_profile, encoding='shift-jis', 
#                       usecols=categorical + ['モニターID(11桁)'])\
#                 .rename(columns=columns_jp2en)\
#                 .set_index('Monitor_ID')

# for col_nm, func_nm in zip(categorical, func_list):
#     if func_nm:
#         df_cate[col_nm] = df_cate[col_nm].map(func_nm)
    
# df_profile = df_cate.join(df_dummy, on='Monitor_ID')

# print(df_cate.shape)
# print(df_profile.shape)
# df_profile

In [5]:
file_ls = sorted(os.listdir(log_root))
usecols = [
    'ts_max', 'ts_min', 'ts_diff', 'ts_len', 'ts_density',
    'dr_max', 'dr_min', 'dr_mean', 'dr_median', 'dr_var'
]

for month in range(12):
    path_os = os.path.join(log_root, file_ls[month * 4 + 3])
    df_os = dd.read_csv(path_os, compression='zip')\
                .set_index('Monitor_ID')
#     df_os裡面的dummy variable, iPhone==0, Android==1
    print(path_os, df_os.shape)
    
    for cata in range(3):
        file_nm = file_ls[month * 4 + cata]
        path_login = os.path.join(log_root, file_nm)
        file_nm = file_nm[:-9] + r'_features.csv'
        
        print(path_login, r'computing now ...')
        
        dd.read_csv(path_login, compression='zip', 
                    usecols=usecols + ['Date', 'Monitor_ID'])\
            .drop(columns='Date')\
            .join(df_os, on='Monitor_ID')\
            .join(df_profile, on='Monitor_ID')\
            .compute()\
            .to_csv(os.path.join(out_root, file_nm), 
                    header=True, index=None, compression='zip')
        
        print(f'>> {file_nm} save file completed!')

D:\issp_data\fe_log_new\s35809_MobileApp_201701_104628_os.csv (Delayed('int-66dcb012-9cbc-42bc-abbd-292fffad255e'), 1)
D:\issp_data\fe_log_new\s35809_MobileApp_201701_104628_0logfe.csv computing now ...
>> s35809_MobileApp_201701_104628_0_features.csv save file completed!
D:\issp_data\fe_log_new\s35809_MobileApp_201701_104628_1logfe.csv computing now ...
>> s35809_MobileApp_201701_104628_1_features.csv save file completed!
D:\issp_data\fe_log_new\s35809_MobileApp_201701_104628_2logfe.csv computing now ...
>> s35809_MobileApp_201701_104628_2_features.csv save file completed!
D:\issp_data\fe_log_new\s35809_MobileApp_201702_104645_os.csv (Delayed('int-f8dc3f4a-bc75-4e44-95fd-b796992e3253'), 1)
D:\issp_data\fe_log_new\s35809_MobileApp_201702_104645_0logfe.csv computing now ...
>> s35809_MobileApp_201702_104645_0_features.csv save file completed!
D:\issp_data\fe_log_new\s35809_MobileApp_201702_104645_1logfe.csv computing now ...
>> s35809_MobileApp_201702_104645_1_features.csv save file com

In [6]:
path = r'D:\issp_data\features_total\s35809_MobileApp_201701_104628_0_features.csv'
df_os = dd.read_csv(path, compression='zip').compute()

df_os

,Monitor_ID,ts_max,ts_min,ts_diff,ts_len,ts_density,dr_max,dr_mean,dr_median,dr_min,...,個人年収,1ヶ月平均小遣い,家族人数,家族構成,世帯年収,家屋形態,生協加入有無,同居配偶者年齢(5才刻み),同居配偶者職業,自動車(自家用車保有)有無
0,50000010367,1483658572,1483577852,80720,113,714.336283,6159,125.194690,16.0,1,...,10,4,3,3,5,1,1,9,9,1
1,50000011091,1483348718,1483348602,116,6,19.333333,37,15.166667,13.0,1,...,10,2,4,3,3,3,2,7,4,1
2,50000011091,1484552063,1484551003,1060,15,70.666667,120,26.000000,7.0,1,...,10,2,4,3,3,3,2,7,4,1
3,50000011730,1485381088,1485329765,51323,19,2701.210526,320,55.000000,10.0,10,...,1,2,2,2,1,1,2,9,2,1
4,50000011763,1485209843,1485161078,48765,17,2868.529412,2047,417.176471,186.0,27,...,10,2,3,3,2,3,2,4,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9751,50001489625,1483315020,1483247909,67111,57,1177.385965,463,46.842105,18.0,1,...,10,5,4,3,1,1,1,10,14,2
9752,50001489659,1484265586,1484179734,85852,117,733.777778,1349,95.153846,23.0,1,...,10,3,6,4,4,1,2,10,14,1
9753,50001490208,1485560754,1485503038,57716,37,1559.891892,1879,133.216216,14.0,1,...,6,3,6,4,3,1,2,4,9,1
9754,50001491836,1484683680,1484638959,44721,61,733.131148,3206,121.672131,24.0,1,...,10,2,4,3,1,2,2,2,4,2
